In [1]:
import pandas as pd
import sqlite3

# Connect to the database
temp_path = './database.db'
connection = sqlite3.connect(temp_path)

**1) Create a Python function to read in the files in the <database.db> file and count the number of records and show the schema/columns for each table.**

In [2]:
def get_tables_and_columns(connection):
    # Get the list of tables in the database
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(query, connection)
    table_list = tables['name'].tolist()
    # Get the columns of each table
    for table_name in table_list:
        query = f"SELECT * FROM {table_name};"
        temp_df = pd.read_sql(query, connection)
        print(f'Columns in table `{table_name}`:', temp_df.columns.to_list())
        print('Number of records:', temp_df.shape[0])
        print('-'*90)
get_tables_and_columns(connection)

Columns in table `albums`: ['AlbumId', 'Title', 'ArtistId']
Number of records: 347
------------------------------------------------------------------------------------------
Columns in table `sqlite_sequence`: ['name', 'seq']
Number of records: 10
------------------------------------------------------------------------------------------
Columns in table `artists`: ['ArtistId', 'Name']
Number of records: 275
------------------------------------------------------------------------------------------
Columns in table `customers`: ['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
Number of records: 59
------------------------------------------------------------------------------------------
Columns in table `employees`: ['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']
Number of recor

**a) By viewing the ERD image, do you think this Database is a Snowflake or a Star schema?**

That is a snowflake schema because of the normalization.

**b) How many different Customers are there?**

In [3]:
query = 'SELECT COUNT(DISTINCT CustomerId) AS UniqueCustomerCount FROM customers;'
temp_df = pd.read_sql_query(query, connection)
int(temp_df['UniqueCustomerCount'].iloc[0])

59

**c) Which Genre has the shortest average track length, in minutes?**

In [4]:
query = "SELECT GenreId, Milliseconds FROM tracks;"
temp_df = pd.read_sql_query(query, connection)
temp_df = temp_df.groupby('GenreId')['Milliseconds'].mean()
temp_df = temp_df.apply(lambda x:x/1000/60)
temp_df = temp_df.sort_values().reset_index()
query = "SELECT * FROM genres;"
temp_df1 = pd.read_sql_query(query, connection)
temp_df.merge(temp_df1, left_on='GenreId', right_on='GenreId', how='left').rename(columns={'Milliseconds':'Minutes'}).iloc[0]['Name']

'Rock And Roll'

**d) Using the ‘playlists’ table, can you tell me the name of the artist that appears the
most in playlists?**

In [5]:
#Get tables needed to arrive at answer
query = "SELECT * FROM playlist_track;"
temp_df = pd.read_sql_query(query, connection)
query = "SELECT TrackId, AlbumId FROM tracks;"
temp_df1 = pd.read_sql_query(query, connection)
query = "SELECT AlbumId, ArtistId FROM albums;"
temp_df2 = pd.read_sql_query(query, connection)
query = "SELECT * FROM artists;"
temp_df3 = pd.read_sql_query(query, connection)
#Merge and get result
merge_df = temp_df.merge(temp_df1, left_on='TrackId', right_on='TrackId', how='left')
merge_df = merge_df.merge(temp_df2, left_on='AlbumId', right_on='AlbumId', how='left')
res_df = merge_df.groupby('ArtistId').size().reset_index().sort_values(by=0)
res_df = res_df.merge(temp_df3, left_on='ArtistId', right_on='ArtistId')
res_df.iloc[-1]['Name']

'Iron Maiden'

**e) What is the count of Customers who have bought more than $40 of tracks?**

In [6]:
#Get tables needed to arrive at answer
query = "SELECT InvoiceId, CustomerId FROM invoices;"
temp_df = pd.read_sql_query(query, connection)
query = "SELECT InvoiceLineId, InvoiceId, UnitPrice, Quantity FROM invoice_items;"
temp_df1 = pd.read_sql_query(query, connection)
#Merge and get result
merge_df = temp_df.merge(temp_df1, left_on='InvoiceId', right_on='InvoiceId', how='left')
merge_df['Revenue'] = merge_df['UnitPrice']*merge_df['Quantity']
res_df = merge_df.groupby('CustomerId')['Revenue'].sum()
res_df = res_df[res_df > 40]
res_df.size

14

**f) Which Key did you use the most?**

CustomerId (referenced in two questions)

In [7]:
# Close the connection
connection.close()